# Contents
<pre></pre>
<font size=+1>

[Useful Functions](#Useful-Functions)<pre></pre>
[Data Prep](#Data-Prep)<pre></pre>
[Modeling](#Modeling)<pre></pre>
[Linear Regression](#Linear-Regression)<pre></pre>
[Validate Linear Regression](#Validate-Linear-Regression)<pre></pre>
[Logistic Regression](#Logistic-Regression)<pre></pre>
[Validate Logistic Regression](#Validate-Logistic-Regression)<pre></pre>
[Random Forest](#Random-Forest)<pre></pre>

</font>

# Useful Functions

## Function to Magnify any Dataframe

In [ ]:
from IPython.display import HTML

def hover(hover_color="#ffff99"):
    return dict(selector="tr:hover",
                props=[("background-color", "%s" % hover_color)])

def magnify_df(df):
    styles = [
    dict(selector="th",props=[("font-size", "15pt")]),
    dict(selector="td",props=[('padding', "0em 0em")]),
    dict(selector="th:hover",props=[("font-size", "15pt")]),
    dict(selector="tr:hover td:hover",props=[('max-width', '200px'),('font-size', '15pt')])
    ]
    return(df.style.set_table_styles(styles).set_caption("Hover to highlight."))

## Function to Create Correlation Plot

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np   
sns.set(rc={'figure.figsize':(10,13)}, font_scale = 2)

def correlation_plot(df, target_col_name):
    corr = df.loc[:, df.columns != target_col_name].corr()
    plt.figure(figsize=(20,13))
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True
    sns.set_context("paper", font_scale=2)
    plt.title('Variables of Interest in our Analysis', fontsize=40)
    correlation_plot = sns.heatmap(df.loc[:, df.columns != target_col_name].corr(), annot=False, linewidth=.5, cmap="coolwarm", mask=mask)
    return correlation_plot

## Function to Create Confusion Matrix Visualization

In [9]:
from sklearn.metrics import confusion_matrix

#This creates confusion matrix visualization
import matplotlib.pylab as plt
import itertools
import numpy as np

def plot_confusion_matrix(cm, classes, normalize=False):
    plt.figure(figsize = (5,5))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion matrix')
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('Actual')
    plt.xlabel('Predicted')

# Data Prep
[Go back to the Table of Contents](#Contents)

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

cleaned_data = pd.read_csv('cleaned_data.csv')
cleaned_data.drop('Unnamed: 0', 1, inplace=True)
cleaned_data.rename({'y_yes': 'subscribed_deposit'}, axis=1, inplace=True)
cleaned_data.isna().sum()

subscribed_deposit     0
age                    0
balance                0
duration               0
campaign               0
previous               0
marital_married        0
marital_single         0
job_blue-collar        0
job_entrepreneur       0
job_housemaid          0
job_management         0
job_retired            0
job_self-employed      0
job_services           0
job_student            0
job_technician         0
job_unemployed         0
education_secondary    0
education_tertiary     0
default_yes            0
housing_yes            0
loan_yes               0
dtype: int64

### Code For One Hot Encoding Categorical Variables

In [ ]:
#Get one hot encoded values for independent categorical variables
#merge one hot encoded variables with independent numeric variables

# X_enc = pd.get_dummies(df_clean[['marital', 'job', 'education', 'default', 'housing', 'loan', 'y']], drop_first= True)

# numeric_data = df_clean[['age', 'balance', 'duration', 'campaign', 'previous']]
    
# cleaned_data_merge = pd.merge(numeric_data, X_enc, how = 'left', left_index=True, right_index=True)

### Omitted Categories for one hot encoded variables

In [ ]:
#Marital - Divorced
#Jobs - admin
#Education - primary
#default - no
#housing - no
#loan - no

# Modeling
[Go back to the Table of Contents](#Contents)

### Correlation plot

In [ ]:
correlation_plot(cleaned_data, 'age')

# Linear Regression
[Go back to the Table of Contents](#Contents)

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import numpy as np 
import statsmodels.api as sm 
import pylab as py
from sklearn.model_selection import cross_val_score
import statistics
  

def Basic_Linear_Regression(df, target_col_name, test_size):
    cleaned_data_y = df.loc[:, cleaned_data.columns == target_col_name]
    cleaned_data_x = df.loc[:, cleaned_data.columns != target_col_name]

    x_train, x_test, y_train, y_test = train_test_split(cleaned_data_x, cleaned_data_y, test_size=test_size, random_state= 1254)

    #Regression RMSE
    # Train the model
    lm = LinearRegression()
    lm.fit(x_train, y_train)

    y_pred = lm.predict(x_test)

    # Compute the root-mean-square

    lm = LinearRegression()
    lm.fit(x_train, y_train)

    MSEs = cross_val_score(lm, x_train, y=y_train, scoring='neg_mean_squared_error', cv=int((1/test_size)))
    RMSE = (MSEs*-1)**0.5
    
    print('\n')
    print('Basic Linear Regression Average Training RMSE:')
    print('\n')
    print(statistics.mean(RMSE))
    print('\n')
    print('RMSE for each Training fold:')
    print('\n')
    print(RMSE)
    print('\n')
    # Compute the root-mean-square
    RMSE_testing = np.sqrt(mean_squared_error(y_test, y_pred))
    print('\n')
    print('Basic Linear Regression Tested RMSE Score:')
    print('\n')
    print(RMSE_testing)
    print('\n')
    
    #Regression Model
    x_train = sm.add_constant(x_train)# adding a constant
    x_test = sm.add_constant(x_test)
    model=sm.OLS(y_train, x_train).fit()
    print(model.summary())
    
    pvalues_df = pd.DataFrame(model.pvalues)
    pvalues_df.reset_index(inplace=True)
    pvalues_df.rename(columns={"index": "Variables", 0: "Pvalues"}, inplace=True)

    #Creates new dataframe for all significant variables
    df_not_significant = pvalues_df.loc[pvalues_df['Pvalues'] > .05]
    df_not_significant['Pvalues'] = df_not_significant['Pvalues'].apply(lambda x: '%.5f' % x)
    print('\n')
    print('These Are the Linear Regression Variables That are not significant:')
    print('\n')
    print(df_not_significant)
    
    coefficients_df = pd.DataFrame(model.params)
    coefficients_df.reset_index(inplace=True)
    coefficients_df.rename(columns={"index": "Variables", 0: "Absolute_Coefficients"}, inplace=True)
    coefficients_df['Absolute_Coefficients'] = abs(coefficients_df['Absolute_Coefficients'])
    coefficients_df.sort_values(by=["Absolute_Coefficients"], ascending=False, inplace=True)
    Feature_Importance = sns.barplot(x="Absolute_Coefficients", y="Variables", data=coefficients_df)
    
    print('\n')
    print('Figure Showing Relative Variable Coefficients, Remember Some Are Continuous:')
    print('\n')
    return Feature_Importance
    
    
    print('\n')
    print('This is a plot of the Regression fitting  Test Data:')
    print('\n')
    sns.set(style='ticks')
    sns.regplot(y_test, y=y_pred, scatter_kws={'alpha':0.5});
    plt.title('Prediction Performance for Age')
    plt.ylabel('Age')
    plt.xlabel('Trained Regression')
    py.show() 
    
    # np.random generates different random numbers 
    # whenever the code is executed 
    # Note: When you execute the same code  
    # the graph look different than shown below.
    
    print('\n')
    print('This is a QQplot which tests for a normal distribution of residuals where residuals should follow line:')
    print('\n')
    data_points = model.resid
    sm.qqplot(data_points, line ='45')
    py.show() 

In [ ]:
Basic_Linear_Regression(df=cleaned_data, target_col_name='age', test_size=0.1)

# Validate Linear Regression
[Go back to the Table of Contents](#Contents)

In [3]:
import sklearn
import statsmodels.api as sm
import seaborn as sns
sns.set(rc={'figure.figsize':(10,13)}, font_scale = 2)

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
import numpy as np

def Linear_Regression_Test(df, target_col_name, numeric_column_list, test_size):

    #Split Data into training and testing groups
    #We will have to standardize the numeric independent variables    
    cleaned_data_y = df.loc[:, df.columns == target_col_name]
    cleaned_data_x = df.loc[:, df.columns != target_col_name]

    x_train, x_test, y_train, y_test = train_test_split(cleaned_data_x, cleaned_data_y, test_size=test_size, random_state= 1254)

    scalar = sklearn.preprocessing.StandardScaler()

    scalar_x_train_numeric = pd.DataFrame(scalar.fit_transform(x_train[numeric_column_list]), index=x_train[numeric_column_list].index, columns=numeric_column_list)
    scalar_x_test_numeric = pd.DataFrame(scalar.transform(x_test[numeric_column_list]), index = x_test.index, columns=numeric_column_list)

    #Merge standardized numerical independent variables with categorical independent variables
    x_train_complete = pd.merge(scalar_x_train_numeric, x_train[x_train.columns.difference(numeric_column_list)], how = 'left', left_index=True, right_index=True)
    x_test_complete = pd.merge(scalar_x_test_numeric, x_test[x_test.columns.difference(numeric_column_list)], how = 'left', left_index=True, right_index=True)
    
    model = LinearRegression()
    model.fit(x_train_complete, y_train)
    
    lasso_params = {'alpha':np.array([0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000])}
    ridge_params = {'alpha':np.array([0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000])}
    
    Lasso_CV = GridSearchCV(linear_model.Lasso(), 
                            param_grid=lasso_params, scoring='neg_mean_squared_error', cv=int((1/test_size))).fit(x_train, y_train)
    print('Average Training Lasso RMSE Score With best Penalty: ', abs(Lasso_CV.best_score_)**0.5)
    print('\n')

    print('Best Lasso Penalty:', Lasso_CV.best_estimator_.get_params()['alpha'])
    print('\n')
    
    Ridge_CV = GridSearchCV(linear_model.Ridge(), 
                            param_grid=ridge_params, scoring='neg_mean_squared_error', cv=int((1/test_size))).fit(x_train, y_train)
    print('Best Training Ridge RMSE Score With best Penalty: ', abs(Ridge_CV.best_score_)**0.5)
    print('\n')
    print('Best Ridge Penalty:', Ridge_CV.best_estimator_.get_params()['alpha'])
    print('\n')
    
    model_OLS = sm.OLS(y_train, x_train)
    results_fu = model_OLS.fit()

    results_fr = model_OLS.fit_regularized(L1_wt=1.0, alpha=Lasso_CV.best_estimator_.get_params()['alpha'], start_params=results_fu.params)
    results_fr_fit = sm.regression.linear_model.OLSResults(model_OLS, results_fr.params, model_OLS.normalized_cov_params)

    final = sm.regression.linear_model.OLSResults(model_OLS, results_fr.params, model_OLS.normalized_cov_params)
    print('Best Lasso Results:')
    print('\n')
    print(final.summary())

    Best_Lasso_Coefficients = pd.DataFrame(final.params)
    Best_Lasso_Coefficients.reset_index(inplace=True)
    Best_Lasso_Coefficients.rename(columns={"index": "Variables", 0: "Coefficients"}, inplace=True)
    Best_Lasso_Coefficients.sort_values(by=["Coefficients"], ascending=False, inplace=True)

    Feature_Importance_Lasso = sns.barplot(x="Coefficients", y="Variables", data=Best_Lasso_Coefficients)
    return Feature_Importance_Lasso

### Best Lasso Model RMSE, Coefficients, and Feature Importance

In [ ]:
Linear_Regression_Test(cleaned_data, 'age', ['balance', 'campaign', 'duration'], 0.1)

# Logistic Regression
[Go back to the Table of Contents](#Contents)

In [ ]:
#Prints accuracy of normal logistic regression and coefficient values after scaling

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.metrics import roc_auc_score, auc, roc_curve, confusion_matrix

def Basic_Logistic_Regression(cleaned_data, target_col_name, numeric_column_list, test_size):

    #Split Data into training and testing groups
    #We will have to standardize the numeric independent variables    
    cleaned_data_y = cleaned_data.loc[:, cleaned_data.columns == target_col_name]
    cleaned_data_x = cleaned_data.loc[:, cleaned_data.columns != target_col_name]

    x_train, x_test, y_train, y_test = train_test_split(cleaned_data_x, cleaned_data_y, test_size=test_size, random_state= 1254)

    scalar = sklearn.preprocessing.StandardScaler()

    scalar_x_train_numeric = pd.DataFrame(scalar.fit_transform(x_train[numeric_column_list]), index=x_train[numeric_column_list].index, columns=numeric_column_list)
    scalar_x_test_numeric = pd.DataFrame(scalar.transform(x_test[numeric_column_list]), index = x_test.index, columns=numeric_column_list)

    #Merge standardized numerical independent variables with categorical independent variables
    x_train_complete = pd.merge(scalar_x_train_numeric, x_train[x_train.columns.difference(numeric_column_list)], how = 'left', left_index=True, right_index=True)
    x_test_complete = pd.merge(scalar_x_test_numeric, x_test[x_test.columns.difference(numeric_column_list)], how = 'left', left_index=True, right_index=True)
    
    logreg = LogisticRegression()
    logreg.fit(x_train_complete, y_train)
    y_pred = logreg.predict(x_test_complete)
    print('\n')
    print('Accuracy of logistic regression classifier on test set: {:.4f}'.format(logreg.score(x_test_complete, y_test)))
    print('\n')
    
    model = sm.Logit(y_train, x_train_complete)
    result=model.fit()
    print('\n')
    print(result.summary())
    print('\n')
    
    fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(x_test)[:,1])
    i = np.arange(len(tpr)) 
    roc = pd.DataFrame({'fpr' : pd.Series(fpr, index=i),'tpr' : pd.Series(tpr, index = i), '1-fpr' : pd.Series(1-fpr, index = i), 'tf' : pd.Series(tpr - (1-fpr), index = i), 'thresholds' : pd.Series(thresholds, index = i)})
    roc = roc.iloc[(roc.tf-0).abs().argsort()[:1]]
    print('\n')
    print(roc)
    print('\n')
    
    pvalues_df = pd.DataFrame(result.pvalues)
    pvalues_df.reset_index(inplace=True)
    pvalues_df.rename(columns={"index": "Variables", 0: "Pvalues"}, inplace=True)

    #Creates new dataframe for all significant variables
    df_not_significant = pvalues_df.loc[pvalues_df['Pvalues'] > .05]
    df_not_significant['Pvalues'] = df_not_significant['Pvalues'].apply(lambda x: '%.5f' % x)
    print('Not Significant Variables:')
    print('\n')
    print(df_not_significant)
    print('\n')
    cm = confusion_matrix(y_test, y_pred)
    cm_plot = plot_confusion_matrix(cm, ['0', '1'])
    return cm_plot
    
Basic_Logistic_Regression(cleaned_data, 'subscribed_deposit', ['age','balance', 'campaign', 'duration'], test_size= 0.1)

### Optimum Threshhold

# Validate Logistic Regression
[Go back to the Table of Contents](#Contents)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score, auc, roc_curve, confusion_matrix

def Logistic_Regression_Test(penalty_list, target_col_name, numeric_column_list, test_size):
    #Split Data into training and testing groups
    #We will have to standardize the numeric independent variables    
    cleaned_data_y = cleaned_data.loc[:, cleaned_data.columns == target_col_name]
    cleaned_data_x = cleaned_data.loc[:, cleaned_data.columns != target_col_name]

    x_train, x_test, y_train, y_test = train_test_split(cleaned_data_x, cleaned_data_y, test_size=test_size, random_state= 1254)

    scalar = sklearn.preprocessing.StandardScaler()

    scalar_x_train_numeric = pd.DataFrame(scalar.fit_transform(x_train[numeric_column_list]), index=x_train[numeric_column_list].index, columns=numeric_column_list)
    scalar_x_test_numeric = pd.DataFrame(scalar.transform(x_test[numeric_column_list]), index = x_test.index, columns=numeric_column_list)

    #Merge standardized numerical independent variables with categorical independent variables
    x_train_complete = pd.merge(scalar_x_train_numeric, x_train[x_train.columns.difference(numeric_column_list)], how = 'left', left_index=True, right_index=True)
    x_test_complete = pd.merge(scalar_x_test_numeric, x_test[x_test.columns.difference(numeric_column_list)], how = 'left', left_index=True, right_index=True)

    logreg = LogisticRegression()
    grid={"C":np.array(penalty_list), "penalty":["l1"]}
    logreg_cv=GridSearchCV(logreg,grid,cv=5)
    logreg_cv.fit(x_train,y_train)
    
    print('The Best Penalty:', logreg_cv.best_estimator_.get_params()['penalty'])
    print('The Best C:', logreg_cv.best_estimator_.get_params()['C'])

    logreg = LogisticRegression(penalty='l1', C=logreg_cv.best_estimator_.get_params()['C'])
    logreg.fit(x_train, y_train)
    THRESHOLD = 0.5
    preds = np.where(logreg.predict_proba(x_test)[:,1] > THRESHOLD, 1, 0)
    print('Accuracy : {:.4f}'.format(accuracy_score(y_test, preds)))

In [ ]:
Logistic_Regression_Test([0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000], 'subscribed_deposit', ['age','balance', 'campaign', 'duration'], 0.1)

In [ ]:
from sklearn.metrics import accuracy_score

logreg = LogisticRegression(penalty='l1', C=3)
logreg.fit(x_train, y_train)
THRESHOLD = 0.47
preds = np.where(logreg.predict_proba(x_test)[:,1] > THRESHOLD, 1, 0)
print('Accuracy : {:.4f}'.format(accuracy_score(y_test, preds)))

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, fscore, support = precision_recall_fscore_support(y_test, preds, average='macro')

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

In [ ]:
cm_lasso = confusion_matrix(y_test, preds)
plot_confusion_matrix(cm_lasso, classes=['0','1'], normalize=False)

In [ ]:
#Lasso Prediction Probabilities

from matplotlib import pyplot as plt 
import numpy as np  
   
a = logreg.predict_proba(x_test)[:,1]
bins = np.arange(0, 1, 0.005).tolist()
plt.hist(a, bins)
plt.title("histogram") 
plt.show()

In [ ]:
#List of Lasso Logistic Regression Accuracy at all thresholds

accuracy=[]
for i in range(0,100):
    preds = np.where(logreg.predict_proba(x_test)[:,1] > (0.01 * i), 1, 0)
    accuracy.append(accuracy_score(y_test, preds))

plt.bar(list(range(0,100)), accuracy, label="Threshhold Accuracy")
plt.legend()

# The following commands add labels to our figure.
plt.xlabel('Threshold')
plt.ylabel('Accuracy')
plt.title('Vertical Bar chart')

plt.show()
##Highest Accuracy Threshold

print('Threshold for highest accuracy:', accuracy.index(max(accuracy)), 'Accuracy:', max(accuracy))

In [ ]:
##Highest Accuracy Threshold

accuracy.index(max(accuracy)), max(accuracy)

# Random Forest
[Go back to the Table of Contents](#Contents)

In [37]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# random forest model creation
rfc = RandomForestClassifier()
rfc.fit(x_train, y_train)
# predictions
rfc_predict = rfc.predict(x_test)

accuracy1 = accuracy_score(y_test, rfc_predict)
print(accuracy1)

cm2 = confusion_matrix(y_test, clf_predict)
print(cm2)

precision, recall, fscore, support = precision_recall_fscore_support(y_test, rfc_predict, average='macro')

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))

0.8936122857778767
[[3911   84]
 [ 395  103]]
precision: 0.7265288816355677
recall: 0.6352109313348513
fscore: 0.6644647131242306


In [ ]:
from sklearn.base import clone
import seaborn as sns

def RFC_Feature_Importance(model, x_train, y_train, random_state = 42):
    rfc = RandomForestClassifier()
    rfc.fit(x_train, y_train)
    # predictions
    rfc_predict = rfc.predict(x_test)
    benchmark_score = model_clone.score(x_train, y_train)
    print(benchmark_score)
    # list for storing feature importances
    importances = []
    
    # iterating over all columns and storing feature importance (difference between benchmark and new model)
    for col in x_train.columns:
        model2 = RandomForestClassifier()
        model2.fit(x_train.drop(col, axis = 1), y_train)
        drop_col_score = model2.score(x_train.drop(col, axis = 1), y_train)
        importances.append(benchmark_score - drop_col_score)
    
    importances_df = pd.DataFrame({'Features': x_train.columns, 'Importance' : importances})
    importances_df.sort_values(by= ['Importance'], ascending=False, inplace=True)
    importances_df.reset_index(drop=True, inplace=True)
    
    return importances_df

In [30]:
def RFC_Best_Model(target_col_name, numeric_column_list, test_size, param_grid_dict):
    #Split Data into training and testing groups
    #We will have to standardize the numeric independent variables    
    cleaned_data_y = cleaned_data.loc[:, cleaned_data.columns == target_col_name]
    cleaned_data_x = cleaned_data.loc[:, cleaned_data.columns != target_col_name]

    x_train, x_test, y_train, y_test = train_test_split(cleaned_data_x, cleaned_data_y, test_size=test_size, random_state= 1254)

    scalar = sklearn.preprocessing.StandardScaler()

    scalar_x_train_numeric = pd.DataFrame(scalar.fit_transform(x_train[numeric_column_list]), index=x_train[numeric_column_list].index, columns=numeric_column_list)
    scalar_x_test_numeric = pd.DataFrame(scalar.transform(x_test[numeric_column_list]), index = x_test.index, columns=numeric_column_list)

    #Merge standardized numerical independent variables with categorical independent variables
    x_train_complete = pd.merge(scalar_x_train_numeric, x_train[x_train.columns.difference(numeric_column_list)], how = 'left', left_index=True, right_index=True)
    x_test_complete = pd.merge(scalar_x_test_numeric, x_test[x_test.columns.difference(numeric_column_list)], how = 'left', left_index=True, right_index=True)

    clf = GridSearchCV(RandomForestClassifier(), param_grid_dict, cv=2)
    clf.fit(x_train_complete, y_train)
    clf_predict = clf.predict(x_test_complete)
    accuracy = accuracy_score(y_test, clf_predict)               
    return clf.best_params_, accuracy


In [31]:
param_grid_dict = {
    'n_estimators': [100, 500],
    'max_depth':[3,7,12],
    'min_samples_split': [2, 4]
}

RFC_Best_Model('subscribed_deposit', ['age','balance', 'campaign', 'duration'], 0.1, param_grid_dict)

({'max_depth': 12, 'min_samples_split': 4, 'n_estimators': 500},
 0.8945025595370576)